# **Ejercicios de pair programming Módulo 2 Sprint 2**
## **ETL: Lección 1**
### **Extracción**

En el marco de un proyecto de análisis de datos, se requiere realizar una ETL para integrar información de distintas fuentes de datos. La empresa para la que trabajamos cuenta con una base de datos de la informacion sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde el año 1998 hasta el 2022. Quieren enriquecer los datos con información de la generación de energías renovables y no renovables y crear una base de datos más completa. Para esto nos piden que usemos la API de REData la cual nos da mucha información sobre el mercado energético en España. La documentación de la API la encontraréis 

1. Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable. NOTA: nos piden que el código debe estar en una función. Al final deberemos tener un dataframe como el siguiente:

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

In [4]:
df_completo = pd.DataFrame()
for año in range(2011, 2023):
    url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=day"

    response = requests.get(url=url)
    fichero_json = response.json()["included"][0]["attributes"]["values"] + response.json()["included"][0]["attributes"]["values"]
    df=pd.json_normalize(fichero_json)
    df["¿renovable?"] = response.json()["included"][0]["attributes"]["title"]
    df_completo =pd.concat([df_completo, df], axis=0)

In [47]:
    url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=day"

    response = requests.get(url=url)
    fichero_json = response.json()["included"][0]["attributes"]["values"] + response.json()["included"][1]["attributes"]["values"]
    df=pd.json_normalize(fichero_json)
    if 
    df["renovable"] = response.json()["included"][0]["type"]
    df["no_renovable"] = response.json()["included"][1]["type"]
    df

,value,percentage,datetime,renovable,no_renovable
0,182024.9820,0.290121,2011-01-01T00:00:00.000+01:00,Renovable,No renovable
1,265705.8895,0.387418,2011-01-02T00:00:00.000+01:00,Renovable,No renovable
2,249489.5735,0.295983,2011-01-03T00:00:00.000+01:00,Renovable,No renovable
3,270981.7130,0.310937,2011-01-04T00:00:00.000+01:00,Renovable,No renovable
4,379833.0150,0.452824,2011-01-05T00:00:00.000+01:00,Renovable,No renovable
...,...,...,...,...,...
725,616884.3980,0.800172,2011-12-27T00:00:00.000+01:00,Renovable,No renovable
726,571869.2935,0.741124,2011-12-28T00:00:00.000+01:00,Renovable,No renovable
727,483740.3770,0.612477,2011-12-29T00:00:00.000+01:00,Renovable,No renovable
728,512509.8810,0.656170,2011-12-30T00:00:00.000+01:00,Renovable,No renovable


2.  Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario: